In [1]:
import warnings
import os

import math

import numpy as np
import pandas as pd

import plotly.io as pio
pio.renderers.default = "notebook"
import plotly.express as px
import seaborn as sns

from utils.build_data_modules import *
from utils.build_data_modules_old import *

warnings.filterwarnings('ignore')

%load_ext autoreload
%autoreload 2

In [2]:
version = '0.09'
clases = 3
periodos_modelo = 20  #1 año
meses_max_animales = 12 * 10 # 10 años, impacta en reproductoras
ventas_max_por_mes = 100

#este valor se usa para tomar un stock inicial
#y para pasar precios desde el periodo 0 a el final del modelo
fecha_inicio = '30/06/2020'


print(f'''
clases: {clases} 
años modelo: {float(periodos_modelo / 12 )} años 
meses vida max: {meses_max_animales} meses, {int(meses_max_animales/12)} años
ventas max por mes {ventas_max_por_mes}
''')




clases: 3 
años modelo: 1.6666666666666667 años 
meses vida max: 120 meses, 10 años
ventas max por mes 100



### Parametros

In [3]:
path_to_file_parametros = 'parametros.dat'

if os.path.exists(path_to_file_parametros):
    os.remove(path_to_file_parametros)

write_line_to_file(path_to_file_parametros, 'parametros\n')
write_line_to_file(path_to_file_parametros, f'tiempo {periodos_modelo}\n')
write_line_to_file(path_to_file_parametros, f'semanas {meses_max_animales}\n')
write_line_to_file(path_to_file_parametros, f'{ventas_max_por_mes}\n')

# Extra - semanas que son agosto
# nacimientos solo en agosto
path_to_file_agosto = 'agosto_si.dat'
path_to_file_not_agosto = 'agosto_no.dat'
path_to_file_momentos_venta = 'momentos_venta_no_posible_c1_c2.dat'
path_to_file_ventas_max_por_mes = 'ventas_max_por_mes.dat'

if os.path.exists(path_to_file_agosto):
    os.remove(path_to_file_agosto)

if os.path.exists(path_to_file_not_agosto):
    os.remove(path_to_file_not_agosto)


num = 8
counter = 0
for semanas in range(periodos_modelo + 1):

    if counter != num:
        write_line_to_file(path_to_file_not_agosto, f'{counter}\n')
    
    else:
        write_line_to_file(path_to_file_agosto, f'{counter}\n')
        num = num +  8
    
    counter = counter + 1

if os.path.exists(path_to_file_momentos_venta):
    os.remove(path_to_file_momentos_venta)

venta_destete = [6,7,8]
venta_novillo_vaquillona = [16,17,18]
venta_pesados = [30,31,32,33,34,35,36]
venta_all = venta_destete + venta_novillo_vaquillona + venta_pesados

for mes in [x for x in range(0,meses_max_animales) if x not in venta_all]:
    write_line_to_file(path_to_file_momentos_venta, f'{mes}\n')

### Costos

In [4]:
collect_costos = []

#path_to_file_costos = 'modelos/mod_{version}/costos.dat'.format(version=version)
path_to_file_costos = 'costos.dat'

if os.path.exists(path_to_file_costos):
    os.remove(path_to_file_costos)

for periodo, edad_animal, clase in itertools.product(range(1,periodos_modelo+1), range(-1,meses_max_animales + 1), range(1,clases+1)):

    # a mayor edad del animal, el impacto es positivo pero decreciente en funcion del tiempo
    # a medida que avanzan los periodos, es menos costoso mantener a un animal
    costo = 1 + edad_animal * 0.25
    
    if math.isinf(costo) | math.isnan(costo) :
        costo = 1
    #else:
    if clase == 3:
        costo = 0

    valores = [periodo,edad_animal,clase,costo,'\n']
    line = '\t'.join(str(x) for x in valores)
    write_line_to_file(path_to_file_costos, line)

    #plot costos
    collect_costos.append({'periodo': periodo , 'costo': costo, 'edad':edad_animal, 'clase':clase})

df_costos = pd.DataFrame(collect_costos)


### plot costos pending

In [5]:
# heatmap_c1 = df_costos.loc[df_costos.clase == 1].pivot(index='edad', columns='periodo', values=['costo'])
# heatmap_c2 = df_costos.loc[df_costos.clase == 2].pivot(index='edad', columns='periodo', values=['costo'])
# heatmap_c3 = df_costos.loc[df_costos.clase == 3].pivot(index='edad', columns='periodo', values=['costo'])


# #df = px.data.medals_wide(indexed=True)
# fig = px.imshow(heatmap_c1)
# fig.show()

# fig = px.imshow(heatmap_c2)
# fig.show()

# fig = px.imshow(heatmap_c3)
# fig.show()

# df_costos_b = df_costos.loc[df_costos['edad'] == 100]
# fig = px.line(df_costos_b, x="periodo", y="costo", color='clase')
# fig.show()

### PRECIOS Y STOCK INICIAL V1 reales

In [7]:
%load_ext autoreload
%autoreload 2
from utils.build_data_modules import *

FECHA_PARTE_DIARIO = '6/26/2020'
FECHA_INICIO_SCRAPPING = '30/06/2020'

precio_prom =  {
              'peso_prom_destete': 164, 
              'peso_prom_vaquillonas': 314,
              'peso_prom_novillitos': 367,
              'peso_prom_vaquillonas_pesados': 433,
              'peso_prom_novillos_pesados': 438
       }

df_precios = get_precios_scrapped(fecha_inicio = FECHA_INICIO_SCRAPPING,
                    input='data/scrapping_df3 - scrapping_df3.csv')

#plot_precios(df_precios)

a = precios_scrapped_to_dat(df_precios,
                        periodos_modelo = periodos_modelo,
                        clases=clases,
                        meses_max_animales = meses_max_animales,
                        peso_prom_dict =precio_prom,
                        path_to_file_precios= 'precios.dat',
                        fecha_inicio = '30/06/2020'
)


get_stock_inicial_from_parte_diario(FECHA_PARTE_DIARIO,
                                        clases,
                                        meses_max_animales,
                                        parte_diario_path='data/datos experimento tesis  - parte_diario completo.csv',
                                        output='stock_inicial.dat'
                                        )

df_precios.to_csv('precios.csv')                                 

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
df_collect = pd.DataFrame()
with open('precios_missing_16062022_150123.txt', 'r') as f:
    capture_lines = False
    for line in f:
        line = line.strip()  # Remove leading/trailing whitespace
        if line == "Fecha Inicial":
            capture_lines = True
            count = 0
            continue  # Start reading the next line
        if capture_lines:
            if count < 50:  # Capture only the next 50 lines
                # Check if the line meets the conditions

                if count == 0:
                    fecha = line

                if 'NOVILLOS Esp.Joven' in line:
                    novillos_exp_joven_avg = int(line.split(' ')[8].split(',')[0])
                
                if 'NOVILLOS Regular h 430' in line:
                    novillos_regular_avg = int(line.split(' ')[8].split(',')[0])
                
                count += 1
            else:
                serie = pd.Series([fecha, novillos_exp_joven_avg, novillos_regular_avg], index=['fecha', 'novillos_exp_joven_avg', 'novillos_regular_avg'])
                #print(fecha, novillos_exp_joven_avg, novillos_regular_avg)
                df_collect = pd.concat([df_collect, serie.to_frame().T])
                capture_lines = False

df_collect

,fecha,novillos_exp_joven_avg,novillos_regular_avg
0,16/06/2022,293,255
0,30/06/2022,285,260
0,15/07/2022,290,255
0,31/07/2022,302,272
0,15/08/2022,305,270
0,31/08/2022,310,270
0,15/09/2022,300,270
0,30/09/2022,293,260
0,15/10/2022,280,247
0,31/10/2022,280,254


### Precios y stock inicial V0 test

In [8]:
# get_stock_inicial_v0(output_path= 'stock_inicial.dat',
#                         clases= clases,
#                         meses_max_animales= meses_max_animales,
#                         edad_animal= meses_max_animales)

# get_precios_v0(output_path = 'precios.dat',
#                 clases= clases,
#                 meses_max_animales= meses_max_animales,
#                 edad_animal= meses_max_animales,
#                 periodos_modelo= periodos_modelo)

### Ventas de parte diario a pesos usando precios scrapped

In [9]:
df_ventas = get_ventas_inicial_from_parte_diario(parte_diario_path='data/datos experimento tesis  - parte_diario completo.csv')


# iteramos por cada venta en el parte diario y la pesificamos usando el precio mas cercano
# en el dataset de precios scrappeados. Obtenemos una nueva columna en df_ventas con la venta en pesos para ese dia

for index, row in df_ventas.iterrows():
    precios = get_precios_del_periodo(row.FECHA, df_precios)
    sub_tot_VAQUILLONAS270 = row['VAQUILLONAS270'] * precios['VAQUILLONAS270']
    sub_tot_NOVILLITOS391 = row['NOVILLITOS391'] * precios['NOVILLITOS391']
    sub_tot_NOVILLITOS300 = row['NOVILLITOS300'] * precios['NOVILLITOS300']
    #sub_tot_VACAS = row['VACAS'] * precios['VACAS']
    #sub_tot_TOROS = row['TOROS'] * precios['TOROS']
    #sub_tot_TERNEROS_DESTETE = row['TERNEROS_DESTETE'] * precios['TERNEROS_DESTETE']
    #sub_tot_TERNERAS_DESTETE = row['TERNERAS_DESTETE'] * precios['TERNERAS_DESTETE']
    VENTA_TOT_DIA = (sub_tot_VAQUILLONAS270 + 
        sub_tot_NOVILLITOS391 +
        sub_tot_NOVILLITOS300 
    #    sub_tot_VACAS +
    #    sub_tot_TOROS +
    #    sub_tot_TERNEROS_DESTETE +
    #    sub_tot_TERNERAS_DESTETE
    )
    df_ventas.loc[index, 'VENTA_PESOS'] = VENTA_TOT_DIA